# Function Calling
The recent fine tuning of the openai has added the additional arguments call functions by which the users can pass descriptions of the functions.

# SetUp the Environment

In [1]:
import os  
import openai 
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
load_dotenv(Path("raj.env"))
openai.api_key=os.getenv("OPENAI_API_KEY")

# Write the Api Call Function

This is an external function that can be used to generate better result for the case gpt model does not have access to  the information.

 Example dummy function hard coded to return the same weather
In production, this could be your backend API or an external API



In [2]:
import json
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

# Define the Function

In [3]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

# Call OpenAi

In [4]:
#!pip install -U openai
from openai import OpenAI
client = OpenAI()

# Call with Functions


Here make the function call but notice that the output content is None but it returns the fnction call arguments which can be later used to call the get_weather function to get the results. 

In [5]:
## This is a list of message to pass to the language model.
## Notice the first line of the message it is to ensure that response to the model is in JSON format.

messages = [
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

print(response)
print("The conent of the message is :")
print(response.choices[0].message.content)
print("the function call is:")
print(response.choices[0].message.function_call)

ChatCompletion(id='chatcmpl-8oFMoiWwO0AuU7OWaiU3VljjKxf6Y', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1706986162, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=92, total_tokens=110))
The conent of the message is :
None
the function call is:
FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather')


In [6]:
args=json.loads(response.choices[0].message.function_call.arguments)
get_current_weather(args)

'{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

# Function call with content ( no weather)

In [7]:

## This has notihing to the above function
messages = [
    {"role": "system",
     "content": "You are a helpful assistant designed to output JSON."},
    {
        "role": "user",
        "content": "hi!",
    }
]

response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)
print(response)
print("The conent of the message is :")
print(response.choices[0].message.content)
print("the function call is:")
print(response.choices[0].message.function_call)
## Check the function call which is None.

ChatCompletion(id='chatcmpl-8oFMprNyaKNSX1cFvghSFbXBCmosf', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1706986163, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=86, total_tokens=96))
The conent of the message is :
Hello! How can I assist you today?
the function call is:
None


# Function call auto (let openai choose)

In [8]:
## We are using auto as the call.
## As the message has nothing to do with the weather it is responding as usual

messages = [
    {"role": "system", 
     "content": "You are a helpful assistant designed to output JSON."},
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response)
print("The conent of the message is :")
print(response.choices[0].message.content)
print("the function call is:")
print(response.choices[0].message.function_call)

ChatCompletion(id='chatcmpl-8oFMpl8bTgnbU4TooUIdiuCEOgsjK', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1706986163, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=86, total_tokens=96))
The conent of the message is :
Hello! How can I assist you today?
the function call is:
None


# Function call is None

In [9]:
## Here the function call is None. So No function is being called.

messages = [
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {
        "role": "user",
        "content": "hi!. Answer in JSON format",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none"
)
print(response)
print("The conent of the message is :")
print(response.choices[0].message.content)
print("the function call is:")
print(response.choices[0].message.function_call)

ChatCompletion(id='chatcmpl-8oFMqYPuW9RXG6alGTd5tHieJz4eO', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1706986164, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=91, total_tokens=100))
The conent of the message is :
Hello! How can I assist you today?
the function call is:
None


# Function call with get_current_weather

In [10]:
messages = [
    {"role": "system", 
     "content": "You are a helpful assistant designed to output JSON."},
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)
print("The conent of the message is :")
print(response.choices[0].message.content)
print("the function call is:")
print(response.choices[0].message.function_call)

ChatCompletion(id='chatcmpl-8oFMrus4kCXtzxEbEy7rWPsbv8q2z', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1706986165, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=11, prompt_tokens=99, total_tokens=110))
The conent of the message is :
None
the function call is:
FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather')


# Append the call and call function with args

In [11]:
message_dict={
    "role": response.choices[0].message.role,
    "content" : response.choices[0].message.content,
    "function_call" : {
       "name": response.choices[0].message.function_call.name,
        "arguments" : response.choices[0].message.function_call.arguments
    }
}

message_dict_json= json.dumps(message_dict)
messages.append(json.loads(message_dict_json))

args = json.loads(response.choices[0].message.function_call.arguments)
observation = get_current_weather(args)

print(messages)




messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)


response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)
print(response)

print( response.choices[0].message.content)

[{'role': 'system', 'content': 'You are a helpful assistant designed to output JSON.'}, {'role': 'user', 'content': "What's the weather like in Boston!"}, {'role': 'assistant', 'content': None, 'function_call': {'name': 'get_current_weather', 'arguments': '{\n  "location": "Boston, MA"\n}'}}]
ChatCompletion(id='chatcmpl-8oFMrrUk6fqiibvZApvHAHcxvlbER', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current weather in Boston, MA is 72 degrees Fahrenheit. It is sunny and windy.', role='assistant', function_call=None, tool_calls=None))], created=1706986165, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=19, prompt_tokens=91, total_tokens=110))
The current weather in Boston, MA is 72 degrees Fahrenheit. It is sunny and windy.


# Notes: 

The idea is to use different api or function to incorporate in the api call. Some simple rule to follow: 
1. Create an api get_current_weather.
2. Write a function for function call ( as described above)
3. Call openai with message acrording to the api and use functions= functions and function_call = {} and save response. 
4. Use the response to parse 
            {role:
             content:
             argumnets:
             }
   and append it calling message 
   messages = [
    {"role": "system", 
     "content": "You are a helpful assistant designed to output JSON."},
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]

It converted to 
[{'role': 'system',
  'content': 'You are a helpful assistant designed to output JSON.'},
 {'role': 'user', 'content': "What's the weather like in Boston!"},
 {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'get_current_weather',
   'arguments': '{\n  "location": "Boston, MA"\n}'}}
   ]

5. Next call the function get_current_weather and append it to the message: 
args = json.loads(response.choices[0].message.function_call.arguments)
observation = get_current_weather(args)

messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)

print( response.choices[0].message.content)

The current weather in Boston, MA is 72 degrees Fahrenheit with a sunny and windy forecast.



As we are using function and function call in the message the number of tokens increases. So we need to account for that in our call. 